In [1]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import TreebankWordTokenizer
import json
import nltk
import os, sys, json, argparse
import pandas as pd
import numpy as np
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from string import punctuation

import urllib.request, html5lib
from bs4 import BeautifulSoup, Tag, UnicodeDammit
import re
from nltk.tokenize.api import TokenizerI
from nltk.tokenize.util import align_tokens

# import punkt
from nltk.tokenize import TreebankWordTokenizer
import nltk.tokenize.punkt
from nltk.probability import FreqDist

In [2]:
SWdf = pd.read_csv('NLPout.csv')

In [3]:
d = {"BEN'S VOICE":"BEN","LUKE'S VOICE":'LUKE',"HAN'S VOICE":"HAN", "FIRST TROOPER":"TROOPER", "TROOPER VOICE":"TROOPER"}
SWdf = SWdf.replace(d)

In [4]:
chars = chars = [ 'LEIA', 'BEN','LUKE','HAN','TROOPER','VADER','WEDGE','THREEPIO','TARKIN','RED LEADER','OFFICER','JABBA','BIGGS']

ccorpus = SWdf.loc[SWdf['char'].isin(chars)]



ccorp = ccorpus.groupby('char')['text'].apply(' '.join).reset_index()

In [5]:
#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
from sklearn.model_selection import train_test_split

train, test = train_test_split(ccorp, test_size=0.2)



In [6]:
print(train)

          char                                               text
4         LEIA  lord vader, i should have known. only you coul...
7   RED LEADER  are you... luke skywalker? have you been check...
9     THREEPIO  did you hear that? they've shut down the main ...
8       TARKIN  the imperial senate will no longer be of any c...
11       VADER  where are those transmissions you intercepted?...
1        BIGGS  just now. i wanted to surprise you, hot shot. ...
5         LUKE  hurry up! come with me! what are you waiting f...
3        JABBA  come on out, solo! i expected you would be. (f...
0          BEN  hello there! come here my little friend. don't...
2          HAN  han solo. i'm captain of the millennium falcon...


In [7]:


# Make a new Tokenizer
stokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()


# Train tokenizer
#stokenizer.train(ylist)
 
wtokenizer = TreebankWordTokenizer()
#https://stackoverflow.com/questions/33098040/how-to-use-word-tokenize-in-data-frame

ccorp['para'] = ccorp.apply(lambda row: stokenizer.tokenize(row['text']), axis=1)





In [8]:
ccorp['sent'] = ccorp.para.apply(lambda x: list(map(str, x)), 1).str.join(',')

In [9]:
ccorp['word'] = ccorp.apply(lambda row: wtokenizer.tokenize(row['sent']), axis=1)

ccorp['char'] = ccorp['char'].replace('RED LEADER', 'RED_LEADER')

ccorp

,char,text,para,sent,word
0,BEN,hello there! come here my little friend. don't...,"[hello there!, come here my little friend., do...","hello there!,come here my little friend.,don't...","[hello, there, !, ,, come, here, my, little, f..."
1,BIGGS,"just now. i wanted to surprise you, hot shot. ...","[just now., i wanted to surprise you, hot shot...","just now.,i wanted to surprise you, hot shot.,...","[just, now., ,, i, wanted, to, surprise, you, ..."
2,HAN,han solo. i'm captain of the millennium falcon...,"[han solo., i'm captain of the millennium falc...","han solo.,i'm captain of the millennium falcon...","[han, solo., ,, i, 'm, captain, of, the, mille..."
3,JABBA,"come on out, solo! i expected you would be. (f...","[come on out, solo!, i expected you would be.,...","come on out, solo!,i expected you would be.,(f...","[come, on, out, ,, solo, !, ,, i, expected, yo..."
4,LEIA,"lord vader, i should have known. only you coul...","[lord vader, i should have known., only you co...","lord vader, i should have known.,only you coul...","[lord, vader, ,, i, should, have, known., ,, o..."
5,LUKE,hurry up! come with me! what are you waiting f...,"[hurry up!, come with me!, what are you waitin...","hurry up!,come with me!,what are you waiting f...","[hurry, up, !, ,, come, with, me, !, ,, what, ..."
6,OFFICER,to you stations! (to another officer) come wit...,"[to you stations!, (to another officer) come w...","to you stations!,(to another officer) come wit...","[to, you, stations, !, ,, (, to, another, offi..."
7,RED_LEADER,are you... luke skywalker? have you been check...,"[are you... luke skywalker?, have you been che...","are you... luke skywalker?,have you been check...","[are, you, ..., luke, skywalker, ?, ,, have, y..."
8,TARKIN,the imperial senate will no longer be of any c...,[the imperial senate will no longer be of any ...,the imperial senate will no longer be of any c...,"[the, imperial, senate, will, no, longer, be, ..."
9,THREEPIO,did you hear that? they've shut down the main ...,"[did you hear that?, they've shut down the mai...","did you hear that?,they've shut down the main ...","[did, you, hear, that, ?, ,, they, 've, shut, ..."


In [10]:
ccorpw = dict(zip(ccorp.char,ccorp.word))
#ccorpw

In [11]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
xwords = []
#xwords = stopwords.words('english')
xwords.extend([",","'s","(",")","...","'re","!","?","n't","'ll","'' ","'","--",";",".","''","``","get","'m","'ve","'d"])
#topWords = set(swords.words('english'))

xwords

[',',
 "'s",
 '(',
 ')',
 '...',
 "'re",
 '!',
 '?',
 "n't",
 "'ll",
 "'' ",
 "'",
 '--',
 ';',
 '.',
 "''",
 '``',
 'get',
 "'m",
 "'ve",
 "'d"]

In [12]:
#choose how the token is processed
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
from nltk.stem import PorterStemmer
#function for removing stopwords
ps = PorterStemmer()
def swords(v):
    words = []
    for w in v:
        if w not in xwords:
            words.append(ps.stem(w))
    return words

In [13]:
#use the function on the data
cdist = dict((k, swords(v)) for k, v in ccorpw.items())


        


In [14]:
keys, values = zip(*cdist.items())

values

(['hello',
  'there',
  'come',
  'here',
  'my',
  'littl',
  'friend.',
  'do',
  'be',
  'afraid.',
  'do',
  'worri',
  'he',
  'be',
  'all',
  'right.',
  'rest',
  'easi',
  'son',
  'you',
  'had',
  'a',
  'busi',
  'day.',
  'you',
  'fortun',
  'you',
  'still',
  'in',
  'one',
  'piece.',
  'the',
  'jundland',
  'wast',
  'are',
  'not',
  'to',
  'be',
  'travel',
  'lightly.',
  'tell',
  'me',
  'young',
  'luke',
  'what',
  'bring',
  'you',
  'out',
  'thi',
  'far',
  'obi-wan',
  'kenobi',
  'obi-wan',
  'now',
  'that',
  'a',
  'name',
  'i',
  'have',
  'heard',
  'in',
  'a',
  'long',
  'time',
  'a',
  'long',
  'time.',
  'oh',
  'he',
  'not',
  'dead',
  'not',
  'not',
  'yet.',
  'well',
  'of',
  'cours',
  'of',
  'cours',
  'i',
  'know',
  'him.',
  'he',
  'me',
  'i',
  'have',
  'gone',
  'by',
  'the',
  'name',
  'obi-wan',
  'sinc',
  'oh',
  'befor',
  'you',
  'were',
  'born.',
  'do',
  'seem',
  'to',
  'rememb',
  'ever',
  'own',
  'a',

In [15]:
from collections import Counter, defaultdict

def create_dist(text):
    c = Counter(text)
    print("this is c", c)
    least_common = c.most_common()[-1][1]
    total = sum(c.values())
    #print(least_common)
    #print(total)
    
    for k, v in c.items():
        #print("this is k", k)
        c[k] = v/total

    return defaultdict(lambda: min(c.values()), c)

In [16]:
from collections import Counter, defaultdict
def filex(char):
    for k,v in cdist.items():
        fname = k
        x = cdist[k]
        y = create_dist(x)
        print("this is y", y)
        return y

In [17]:
BEN = dict(filex('BEN'))

len(BEN)

this is c Counter({'the': 47, 'you': 38, 'a': 30, 'your': 21, 'to': 19, 'i': 17, 'be': 16, 'and': 16, 'it': 16, 'of': 15, 'he': 14, 'have': 13, 'as': 10, 'forc': 10, 'can': 10, 'do': 9, 'in': 9, 'are': 9, 'thi': 9, 'that': 9, 'we': 9, 'will': 8, 'with': 8, 'jedi': 8, 'not': 7, 'me': 7, 'luke': 7, 'but': 7, 'on': 7, 'if': 7, 'they': 6, 'for': 6, 'let': 6, 'luke.': 6, 'there': 5, 'what': 5, 'out': 5, 'by': 5, 'wa': 5, 'is': 5, 'must': 5, 'too': 5, 'come': 4, 'littl': 4, 'obi-wan': 4, 'now': 4, 'well': 4, 'befor': 4, 'were': 4, 'you.': 4, 'father': 4, 'old': 4, 'more': 4, 'onli': 4, 'imperi': 4, 'go': 4, 'here': 3, 'my': 3, 'friend.': 3, 'all': 3, 'name': 3, 'long': 3, 'time': 3, 'rememb': 3, 'think': 3, 'uncl': 3, 'should': 3, 'someth': 3, 'it.': 3, 'or': 3, 'an': 3, 'who': 3, 'hi': 3, 'see': 3, 'ca': 3, 'alderaan.': 3, 'need': 3, 'these': 3, 'alway': 3, 'could': 3, 'droid': 3, 'no': 3, 'son': 2, 'had': 2, 'one': 2, 'young': 2, 'far': 2, 'time.': 2, 'oh': 2, 'cours': 2, 'him.': 2, 'seem'

427

In [18]:
BEN = (filex('BEN'))
BIGGS = dict(filex('BIGGS'))
HAN = dict(filex('HAN'))
JABBA = dict(filex('JABBA'))
LEIA = dict(filex('LEIA'))
LUKE = dict(filex('LUKE'))
OFFICER = dict(filex('OFFICER'))
RED_LEADER = dict(filex('RED_LEADER'))
TARKIN = dict(filex('TARKIN'))
THREEPIO = dict(filex('THREEPIO'))
TROOPER = dict(filex('TROOPER'))
VADER = dict(filex('VADER'))
WEDGE = dict(filex('WEDGE'))



this is c Counter({'the': 47, 'you': 38, 'a': 30, 'your': 21, 'to': 19, 'i': 17, 'be': 16, 'and': 16, 'it': 16, 'of': 15, 'he': 14, 'have': 13, 'as': 10, 'forc': 10, 'can': 10, 'do': 9, 'in': 9, 'are': 9, 'thi': 9, 'that': 9, 'we': 9, 'will': 8, 'with': 8, 'jedi': 8, 'not': 7, 'me': 7, 'luke': 7, 'but': 7, 'on': 7, 'if': 7, 'they': 6, 'for': 6, 'let': 6, 'luke.': 6, 'there': 5, 'what': 5, 'out': 5, 'by': 5, 'wa': 5, 'is': 5, 'must': 5, 'too': 5, 'come': 4, 'littl': 4, 'obi-wan': 4, 'now': 4, 'well': 4, 'befor': 4, 'were': 4, 'you.': 4, 'father': 4, 'old': 4, 'more': 4, 'onli': 4, 'imperi': 4, 'go': 4, 'here': 3, 'my': 3, 'friend.': 3, 'all': 3, 'name': 3, 'long': 3, 'time': 3, 'rememb': 3, 'think': 3, 'uncl': 3, 'should': 3, 'someth': 3, 'it.': 3, 'or': 3, 'an': 3, 'who': 3, 'hi': 3, 'see': 3, 'ca': 3, 'alderaan.': 3, 'need': 3, 'these': 3, 'alway': 3, 'could': 3, 'droid': 3, 'no': 3, 'son': 2, 'had': 2, 'one': 2, 'young': 2, 'far': 2, 'time.': 2, 'oh': 2, 'cours': 2, 'him.': 2, 'seem'

In [19]:
import decimal
from decimal import Decimal
decimal.getcontext().prec = 1000

def precise_product(numbers):
    result = 1
    for x in numbers:
        result *= Decimal(x)
    return result

def NaiveBayes(dist):
    """A simple naive bayes classifier that takes as input a dictionary of
    Counter distributions and can then be used to find the probability
    of a given item belonging to each class.
    The input dictionary is in the following form:
        ClassName: Counter"""
    attr_dist = {c_name: count_prob for c_name, count_prob in dist.items()}

    def predict(example):
        """Predict the probabilities for each class."""
        def class_prob(target, e):
            attr = attr_dist[target]
            return precise_product([attr[a] for a in e])

        pred = {t: class_prob(t, example) for t in dist.keys()}

        total = sum(pred.values())
        for k, v in pred.items():
            pred[k] = v / total

        return pred

    return predict

In [20]:
dist = {'BEN' : BEN, 'BIGGS': BIGGS, 'HAN': HAN,  'JABBA': JABBA, 
        'LEIA': LEIA, 'LUKE':LUKE, 'OFFICER':OFFICER, 'RED_LEADER': RED_LEADER,
        'TARKIN':TARKIN, 'THREEPIO':THREEPIO, 'TROOPER':TROOPER,'VADER':VADER,'WEDGE':WEDGE}

dist

{'BEN': defaultdict(<function __main__.create_dist.<locals>.<lambda>()>,
             {'hello': 0.0009784735812133072,
              'there': 0.004892367906066536,
              'come': 0.003913894324853229,
              'here': 0.0029354207436399216,
              'my': 0.0029354207436399216,
              'littl': 0.003913894324853229,
              'friend.': 0.0029354207436399216,
              'do': 0.008806262230919765,
              'be': 0.015655577299412915,
              'afraid.': 0.0009784735812133072,
              'worri': 0.0009784735812133072,
              'he': 0.0136986301369863,
              'all': 0.0029354207436399216,
              'right.': 0.0009784735812133072,
              'rest': 0.0009784735812133072,
              'easi': 0.0009784735812133072,
              'son': 0.0019569471624266144,
              'you': 0.03718199608610567,
              'had': 0.0019569471624266144,
              'a': 0.029354207436399216,
              'busi': 0.00097847358121330

In [21]:
nBS = NaiveBayes(dist)

In [22]:
def recognize(sentence, nBS):
    return nBS(word_tokenize(sentence.lower()))

In [23]:
recognize("The", nBS)

{'BEN': Decimal('0.076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076923076

In [24]:
#find the frequency of each word
cdistc = dict((k, FreqDist(v)) for k, v in cdist.items())
# get a list of unique words per class
cdistu = dict((k, set(v)) for k, v in cdist.items())
#cdistc

In [25]:
#https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra
#Sparse Matrix
cdistdf = pd.DataFrame(cdistc)
cdistdft = cdistdf.T
cdistdft.fillna(0, inplace=True)
cdistdft


,'caus,'em,'em.,'nother,a,a-twenty-,abandon,abid,abil,abl,...,yes.,yet,yet.,you,you.,young,your,yours.,yourself,yourself.
BEN,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,38.0,4.0,2.0,21.0,0.0,0.0,1.0
BIGGS,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
HAN,1.0,1.0,1.0,0.0,36.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,55.0,3.0,0.0,15.0,0.0,3.0,1.0
JABBA,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,12.0,2.0,0.0,4.0,0.0,0.0,0.0
LEIA,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,34.0,1.0,0.0,5.0,0.0,1.0,0.0
LUKE,0.0,0.0,0.0,1.0,43.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106.0,11.0,0.0,8.0,0.0,3.0,1.0
OFFICER,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
RED_LEADER,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,7.0,0.0,1.0,0.0
TARKIN,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,14.0,0.0,0.0,5.0,0.0,0.0,0.0
THREEPIO,0.0,0.0,0.0,0.0,27.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,57.0,3.0,0.0,13.0,0.0,2.0,0.0


In [26]:
from sklearn import preprocessing


scale = preprocessing.MinMaxScaler()
np_scaled = scale.fit_transform(cdistdft)
cdistn = pd.DataFrame(np_scaled)

cdistn




,0,1,2,3,4,5,6,7,8,9,...,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706
0,0.0,0.0,0.0,0.0,0.690476,0.0,0.0,0.0,0.0,1.0,...,0.00,0.0,1.0,0.346154,0.363636,1.0,1.00,0.0,0.000000,1.0
1,0.0,0.0,0.0,0.0,0.238095,0.0,0.0,0.0,0.0,1.0,...,0.00,0.0,0.0,0.192308,0.090909,0.0,0.25,0.0,0.000000,0.0
2,1.0,1.0,1.0,0.0,0.833333,0.0,0.0,0.0,0.0,1.0,...,0.00,1.0,0.0,0.509615,0.272727,0.0,0.70,0.0,1.000000,1.0
3,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,1.0,...,0.00,0.0,0.0,0.096154,0.181818,0.0,0.15,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.214286,0.0,0.0,0.0,0.0,0.0,...,0.00,1.0,0.0,0.307692,0.090909,0.0,0.20,0.0,0.333333,0.0
5,0.0,0.0,0.0,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,1.000000,1.000000,0.0,0.35,0.0,1.000000,1.0
6,0.0,0.0,0.0,0.0,0.047619,0.0,1.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0
7,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.067308,0.000000,0.0,0.30,0.0,0.333333,0.0
8,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,...,1.00,0.0,0.0,0.115385,0.000000,0.0,0.20,0.0,0.000000,0.0
9,0.0,0.0,0.0,0.0,0.619048,1.0,0.0,1.0,0.0,1.0,...,0.00,0.0,0.0,0.528846,0.272727,0.0,0.60,0.0,0.666667,0.0


In [27]:
cols = list(cdistdft.columns.values)

rows = cdistdft.index.values.tolist()

#rows

In [28]:
cdistn.columns = cols
#cdistn.index.names = rows
cdistn

,'caus,'em,'em.,'nother,a,a-twenty-,abandon,abid,abil,abl,...,yes.,yet,yet.,you,you.,young,your,yours.,yourself,yourself.
0,0.0,0.0,0.0,0.0,0.690476,0.0,0.0,0.0,0.0,1.0,...,0.00,0.0,1.0,0.346154,0.363636,1.0,1.00,0.0,0.000000,1.0
1,0.0,0.0,0.0,0.0,0.238095,0.0,0.0,0.0,0.0,1.0,...,0.00,0.0,0.0,0.192308,0.090909,0.0,0.25,0.0,0.000000,0.0
2,1.0,1.0,1.0,0.0,0.833333,0.0,0.0,0.0,0.0,1.0,...,0.00,1.0,0.0,0.509615,0.272727,0.0,0.70,0.0,1.000000,1.0
3,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,1.0,...,0.00,0.0,0.0,0.096154,0.181818,0.0,0.15,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.214286,0.0,0.0,0.0,0.0,0.0,...,0.00,1.0,0.0,0.307692,0.090909,0.0,0.20,0.0,0.333333,0.0
5,0.0,0.0,0.0,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,1.000000,1.000000,0.0,0.35,0.0,1.000000,1.0
6,0.0,0.0,0.0,0.0,0.047619,0.0,1.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0
7,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.067308,0.000000,0.0,0.30,0.0,0.333333,0.0
8,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,...,1.00,0.0,0.0,0.115385,0.000000,0.0,0.20,0.0,0.000000,0.0
9,0.0,0.0,0.0,0.0,0.619048,1.0,0.0,1.0,0.0,1.0,...,0.00,0.0,0.0,0.528846,0.272727,0.0,0.60,0.0,0.666667,0.0


In [29]:
cdistnT = cdistn.T

In [30]:
cdict = cdistnT.to_dict()

cdict

{0: {"'caus": 0.0,
  "'em": 0.0,
  "'em.": 0.0,
  "'nother": 0.0,
  'a': 0.6904761904761904,
  'a-twenty-': 0.0,
  'abandon': 0.0,
  'abid': 0.0,
  'abil': 0.0,
  'abl': 1.0,
  'aboard': 0.0,
  'aboard.': 0.0,
  'about': 0.16666666666666666,
  'about.': 0.0,
  'abov': 0.0,
  'absolut': 0.0,
  'academi': 0.0,
  'academy.': 0.0,
  'acceler': 0.0,
  'accelerator.': 0.0,
  'accord': 0.0,
  'account.': 0.0,
  'accur': 1.0,
  'across': 0.0,
  'across.': 0.0,
  'act': 1.0,
  'action': 0.0,
  'action.': 0.0,
  'actual': 0.0,
  'ad': 0.0,
  'address': 0.0,
  'advance.': 0.0,
  'adventures.': 0.0,
  'advic': 0.0,
  'afraid': 0.0,
  'afraid.': 1.0,
  'after': 0.0,
  'afterburn': 0.0,
  'again': 1.0,
  'again.': 0.0,
  'against': 0.0,
  'against.': 0.0,
  'ago': 0.0,
  'agre': 0.0,
  'agreement': 0.0,
  'ah': 0.0,
  'ahead': 0.0,
  'ahead.': 0.0,
  'ai': 0.0,
  'alderaan': 0.0,
  'alderaan.': 1.0,
  'alert': 0.0,
  'aliv': 0.0,
  'alive.': 0.0,
  'all': 0.15789473684210525,
  'all-pow': 0.0,
  'al

In [31]:
from collections import defaultdict
dist = []
for k1,v1 in cdict.items():
    #print(k1)
    for x, y in v1.items():
        #print(y)
        if float(y) > .6:
            dist.append((x, y))
dist

[('a', 0.6904761904761904),
 ('abl', 1.0),
 ('accur', 1.0),
 ('act', 1.0),
 ('afraid.', 1.0),
 ('again', 1.0),
 ('alderaan.', 1.0),
 ('allow', 1.0),
 ('ally.', 1.0),
 ('alone.', 1.0),
 ('along', 1.0),
 ('also', 1.0),
 ('altern', 1.0),
 ('alway', 1.0),
 ('an', 1.0),
 ('and', 1.0),
 ('as', 1.0),
 ('asked.', 1.0),
 ('avoid', 1.0),
 ('be', 0.8421052631578947),
 ('becom', 1.0),
 ('befor', 1.0),
 ('best', 0.6666666666666666),
 ('betray', 1.0),
 ('bind', 1.0),
 ('blaster.', 1.0),
 ('boast', 1.0),
 ('born.', 1.0),
 ('bring', 1.0),
 ('busi', 1.0),
 ('buy', 1.0),
 ('by', 1.0),
 ('can', 0.6666666666666666),
 ('cautious.', 1.0),
 ('chewbacca.', 1.0),
 ('civil', 1.0),
 ('clumsi', 1.0),
 ('commands.', 1.0),
 ('consciou', 1.0),
 ('cours', 1.0),
 ('course.', 1.0),
 ('creat', 1.0),
 ('cri', 1.0),
 ('crusad', 1.0),
 ('cun', 1.0),
 ('damned-fool', 1.0),
 ('dangerous.', 1.0),
 ('dark', 1.0),
 ('darth', 1.0),
 ('darth.', 1.0),
 ('day.', 1.0),
 ('dead', 1.0),
 ('deceiv', 1.0),
 ('deep', 1.0),
 ('deliv', 1.0

In [32]:
create_dist(dist)

this is c Counter({('there.', 1.0): 6, ('until', 1.0): 6, ('away', 1.0): 6, ('abl', 1.0): 5, ('bring', 1.0): 5, ('dead', 1.0): 5, ('fighter', 1.0): 5, ('short', 1.0): 5, ('star', 1.0): 5, ('next', 1.0): 5, ('time', 0.6666666666666666): 5, ('after', 1.0): 5, ('again', 1.0): 4, ('also', 1.0): 4, ('an', 1.0): 4, ('done', 1.0): 4, ('entir', 1.0): 4, ('first', 1.0): 4, ('hand', 1.0): 4, ('myself', 1.0): 4, ('own', 1.0): 4, ('place', 1.0): 4, ('rest', 1.0): 4, ('safe', 1.0): 4, ('ship.', 1.0): 4, ('stay', 1.0): 4, ('together.', 1.0): 4, ('understand', 1.0): 4, ('which', 1.0): 4, ('aboard', 1.0): 4, ('day', 1.0): 4, ('expect', 1.0): 4, ('sign', 1.0): 4, ('stop', 1.0): 4, ('anoth', 1.0): 4, ('away.', 1.0): 4, ('person', 1.0): 4, ('system.', 1.0): 4, ('life', 1.0): 4, ('allow', 1.0): 3, ('dangerous.', 1.0): 3, ('eisley', 1.0): 3, ('empir', 1.0): 3, ('enough.', 1.0): 3, ('eye', 1.0): 3, ('find', 0.6666666666666666): 3, ('for.', 1.0): 3, ('gone', 1.0): 3, ('good.', 1.0): 3, ('heard', 1.0): 3, ('i

defaultdict(<function __main__.create_dist.<locals>.<lambda>()>,
            {('a', 0.6904761904761904): 0.00041597337770382697,
             ('abl', 1.0): 0.0020798668885191347,
             ('accur', 1.0): 0.00041597337770382697,
             ('act', 1.0): 0.00041597337770382697,
             ('afraid.', 1.0): 0.00041597337770382697,
             ('again', 1.0): 0.0016638935108153079,
             ('alderaan.', 1.0): 0.00041597337770382697,
             ('allow', 1.0): 0.0012479201331114808,
             ('ally.', 1.0): 0.00041597337770382697,
             ('alone.', 1.0): 0.0008319467554076539,
             ('along', 1.0): 0.0008319467554076539,
             ('also', 1.0): 0.0016638935108153079,
             ('altern', 1.0): 0.0008319467554076539,
             ('alway', 1.0): 0.00041597337770382697,
             ('an', 1.0): 0.0016638935108153079,
             ('and', 1.0): 0.0008319467554076539,
             ('as', 1.0): 0.00041597337770382697,
             ('asked.', 1.0): 0.00041

In [33]:
import decimal
from decimal import Decimal
decimal.getcontext().prec = 1000

def precise_product(numbers):
    result = 1
    for x in numbers:
        result *= Decimal(x)
    return result

def NaiveBayes(dist):
    """A simple naive bayes classifier that takes as input a dictionary of
    Counter distributions and can then be used to find the probability
    of a given item belonging to each class.
    The input dictionary is in the following form:
        ClassName: Counter"""
    attr_dist = {c_name: count_prob for c_name, count_prob in dist.items()}
    print(c_name)
    print(count_prob)

    def predict(example):
        """Predict the probabilities for each class."""
        def class_prob(target, e):
            print(target)
            print(e)
            attr = attr_dist[target]
            return precise_product([attr[a] for a in e])

        pred = {t: class_prob(t, example) for t in dist.keys()}

        total = sum(pred.values())
        for k, v in pred.items():
            pred[k] = v / total

        return pred

    return predict

In [34]:
def recognize(sentence, nBS):
    return nBS(word_tokenize(sentence.lower()))

In [35]:
recognize("did you hear that?", nBS)

KeyError: 'hear'

In [ ]:
def create_word_scores(pos_words, neg_words, pos_tag, neg_tag):
    pos_words = list(itertools.chain(*pos_words))
    neg_words = list(itertools.chain(*neg_words))

    word_fd = FreqDist()
    cond_word_fd = ConditionalFreqDist()
    for word in pos_words:
        word_fd[word] += 1
        cond_word_fd[pos_tag][word] += 1
    for word in neg_words:
        word_fd[word] += 1
        cond_word_fd[neg_tag][word] += 1

    pos_word_count = cond_word_fd[pos_tag].N()
    neg_word_count = cond_word_fd[neg_tag].N()
    total_word_count = pos_word_count + neg_word_count

    word_scores = {}
    for word, freq in word_fd.iteritems():
        pos_score = BigramAssocMeasures.chi_sq(cond_word_fd[pos_tag][word], (freq, pos_word_count), total_word_count)
        neg_score = BigramAssocMeasures.chi_sq(cond_word_fd[neg_tag][word], (freq, neg_word_count), total_word_count)
        word_scores[word] = pos_score + neg_score

    return word_scores